# Toy model showing movement through $\Delta$KS - $\Delta$MGMS - $\Delta$SFMS planes with varying fuel and star formation efficiency

In [1]:
# match fonts and font sizes to the aastex paper style
import rsmf

# following http://hea-www.harvard.edu/~alexey/emulateapj/emulateapj.cls
# fix aastex's definition of type sizes where their manuscript 12pt = revtex 10.56pt
formatter = rsmf.setup(
    r"\documentclass[a4paper,10.56pt,noarxiv,amsmath]{revtex4-2}"
)

In [2]:
from IPython.display import Latex, Math, display


def calc_deltas(
    delta_mgms,  # gas density at fixed stellar surface density
    delta_ks,  # sfr density at fixed gas surface density
    sfr_den_main_sequence=-2,  # sfr surface density on SFMS
    mmol_den_main_sequence=1,  # molecular gas surface density on MGMS
    ks_slope=1.0,  # approx KS law best fit (sfr_den = ks_slope * mmol_den + ks_b)
    ks_b=-3,
):

    mmol_den = (
        mmol_den_main_sequence + delta_mgms
    )  # molecular gas surface density
    sfr_den = (mmol_den ** ks_slope) + ks_b + delta_ks  # mol
    delta_sfms = sfr_den - sfr_den_main_sequence  # distance from sfms

    display(Math(r"\Delta MGMS: {:.2f}".format(delta_mgms)))
    display(Math(r"\Delta KS: {:.2f}".format(delta_ks)))
    display(Math(r"N: {:.2f}".format(ks_slope)))

    print("")
    display(Math(r"\Delta SFMS: {:.2f}".format(delta_sfms)))
    return delta_sfms

In [3]:
%matplotlib inline
import numpy as np
from ipywidgets import interactive
from matplotlib.colors import colorConverter
from matplotlib.lines import Line2D


def f(fuel, efficiency, n=1):

    fig = formatter.figure(wide=True, aspect_ratio=1)

    delta_sfms = calc_deltas(delta_mgms=fuel, delta_ks=efficiency, ks_slope=n)

    # set marker color and shape
    if delta_sfms > 0:
        marker_color = "royalblue"
    elif delta_sfms < 0:
        marker_color = "indianred"
    else:
        marker_color = "mediumseagreen"

    if abs(fuel) >= abs(efficiency):
        marker = "o"
    else:
        marker = "s"

    # plot description
    if (delta_sfms >= 0) & (abs(fuel) > abs(efficiency)):
        desc = "star-forming due to increased fuel"
    if (delta_sfms >= 0) & (abs(fuel) < abs(efficiency)):
        desc = "star-forming due to increased efficiency"
    if (delta_sfms < 0) & (abs(fuel) > abs(efficiency)):
        desc = "quiescent due to starvation"
    if (delta_sfms < 0) & (abs(fuel) < abs(efficiency)):
        desc = "quiescent due to inefficiency"
    if (delta_sfms == 0):
        desc = "in equilibrium"
    display(Latex("\nregion is " + desc + "\n"))

    # plot toy model
    ax1 = fig.add_subplot(2, 2, 1)
    ax1.scatter(
        fuel,
        efficiency,
        marker=marker,
        facecolor=colorConverter.to_rgba(marker_color, alpha=0.5),
        edgecolors=marker_color,
        s=500,
    )

    ax2 = fig.add_subplot(2, 2, 3, sharex=ax1)
    ax2.scatter(
        fuel,
        delta_sfms,
        marker=marker,
        facecolor=colorConverter.to_rgba(marker_color, alpha=0.5),
        edgecolors=marker_color,
        s=500,
    )

    ax3 = fig.add_subplot(2, 2, 4, sharey=ax2)
    ax3.scatter(
        efficiency,
        delta_sfms,
        marker=marker,
        facecolor=colorConverter.to_rgba(marker_color, alpha=0.5),
        edgecolors=marker_color,
        s=500,
    )

    # guide lines and limits
    ax1.plot([-2, 2], [2, -2], color="k", alpha=0.5, lw=0.5, zorder=100)
    ax1.plot([-2, 2], [-2, 2], color="k", alpha=0.5, lw=0.5, zorder=100)
    ax1.set_xlim(-1.2, 1.2)
    ax1.set_ylim(-1.2, 1.2)
    ax1.set_yticks([-1, -0.5, 0, 0.5, 1])

    for ax in [ax2, ax3]:
        ax.plot([-2, 2], [0, 0], color="k", alpha=0.5, lw=0.5, zorder=100)
        ax.plot([0, 0], [-2, 2], color="k", alpha=0.5, lw=0.5, zorder=100)
        ax.set_xlim(-1.2, 1.2)
        ax.set_ylim(-1.2, 1.2)
        ax.set_yticks([-1, -0.5, 0, 0.5, 1])
        ax.set_xticks([-1, -0.5, 0, 0.5, 1])

    ax1.tick_params(labelbottom=False)
    ax3.tick_params(labelleft=False)

    ax1.set_ylabel(r"$\Delta$KS")
    ax2.set_ylabel(r"$\Delta$SFMS")
    ax2.set_xlabel(r"$\Delta$MGMS")
    ax3.set_xlabel(r"$\Delta$KS")

    # axes for legend
    ax4 = fig.add_subplot(2, 2, 2)

    # custom legend
    custom_lines = [
        Line2D(
            [0],
            [0],
            linestyle="None",
            marker="None",
        ),
        Line2D(
            [0],
            [0],
            linestyle="None",
            marker="None",
        ),
        Line2D(
            [0],
            [0],
            linestyle="None",
            marker="None",
        ),
        Line2D(
            [0],
            [0],
            linestyle="None",
            marker="o",
            color="gray",
            markerfacecolor="None",
            ms=7,
        ),
        Line2D(
            [0],
            [0],
            linestyle="None",
            marker="s",
            color="gray",
            markerfacecolor="None",
            ms=7,
        ),
    ]

    ax4.legend(
        handles=custom_lines,
        labels=[
            "star-forming",
            "quiescent",
            "equilibrium",
            "fuel-driven",
            "efficiency-driven",
        ],
        labelcolor=["royalblue", "indianred", "mediumseagreen", "k", "k"],
        loc="center",
        # title="VERTICO {}".format(resolution),
        frameon=True,
    )

    ax4.axis("off")

    ax4.axis("off")

    fig.subplots_adjust(hspace=0.05, wspace=0.05)

In [4]:
interactive_plot = interactive(
    f, fuel=(-1, 1, 0.05), efficiency=(-1, 1, 0.05), n=(0.5,1.5,0.1), continuous_update=True
)
output = interactive_plot.children[-1]
# output.layout.height = "950px"
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='fuel', max=1.0, min=-1.0, step=0.05), FloatSlider(va…